
#Introduzione a Pandas

---


*Antonio Emanuele Cinà, Assistant Professor @ University of Genoa*

**Analisi e Rappresentazione dei Dati** --
8 Aprile 2024

## Lezione 5: Pandas


Materiale: https://tinyurl.com/ARD24-L5

# Modificare i dati con le maschere

Inoltre le maschere tornano molto utili quando vogliamo andare a modificare contemporaneamente tutte le righe che rispettano una determinata condizione.

Supponiamo ora di voler contrassegnare tutte le persone che lavorano in **Veneto** con la label **SI** nella colonna **Verificato**, ci basterà creare una maschera che utilizzeremo poi per modificare le righe.


In [ ]:
import pandas as pd
path_dip_csv = "https://github.com/Cinofix/analisi-e-rappresentazione-dati/raw/main/data/dipendenti.csv"

data = pd.read_csv(path_dip_csv)

In [ ]:
maschera_veneto = data['Regione'] == "Veneto"

data.loc[maschera_veneto, "Verificato"].value_counts()

Verificato
NO    21
SI    13
Name: count, dtype: int64

In [ ]:
maschera_veneto = data['Regione'] == "Veneto"

data.loc[maschera_veneto, "Verificato"] = 'SI'

data.loc[maschera_veneto,"Verificato"].value_counts()

Verificato
SI    38
Name: count, dtype: int64

### And, Or e Not

Possiamo anche utilizzare gli operatori `&` (**and**), `|` (**or**) e ~ (**not**) per creare ricerche più sofisticate:

L'esempio seguente mostra come selezionare tutti i dipendenti che **NON** vivono in **Veneto** e neppure in **Toscana**.

In [ ]:
# definisci la maschera con l'operatore OR (|)
veneto_toscana = (data.Regione=="Veneto") | (data.Regione=="Toscana")

# prendi tutti i dati NON nella maschera
data[~veneto_toscana]

In [ ]:
data[ ~((data.Regione=="Veneto") | (data.Regione=="Toscana")) ]

,ID,Nome,Età,Regione,Salario,Verificato
0,829744,Elona,NaN,Sicilia,1921.0,NO
1,800269,Cheikh,53.0,Abruzzo,1563.0,SI
2,823929,Girolama,30.0,Emilia Romagna,2144.0,SI
3,842108,Marwa,60.0,Umbria,1798.0,NO
5,888819,Edda,21.0,Lombardia,2339.0,NO
...,...,...,...,...,...,...
994,848039,Magda,18.0,Emilia Romagna,2353.0,NO
996,870201,Erjon,59.0,NaN,2547.0,NO
997,886127,Carla maria,23.0,Campania,2149.0,NO
998,847566,El mehdi,32.0,NaN,1817.0,NO


### Esercizio 4

1. Prendere il dataset **penguins_size.csv** e calcolare il peso medio dei pinguini maschi

In [ ]:
path_dip_csv = "https://github.com/Cinofix/analisi-e-rappresentazione-dati/raw/main/data/penguins_size.csv"

data = pd.read_csv(path_dip_csv)

only_male = data[data["sex"] == "MALE"]

peso_medio = only_male["body_mass_g"].mean()
peso_medio

4545.684523809524

In [ ]:
peso_medio/1000

4.545684523809524

### Esercizio 5

2. E' un pinguino maschio o un pinguino femmina quello che pesa di più? Qual è la differenza tra i due pesi?

In [ ]:
only_male = data["sex"] == "MALE"
max_male = data[only_male]["body_mass_g"].max()

only_female = data["sex"] == "FEMALE"
max_female = data[only_female]["body_mass_g"].max()

print(f"{max_male=}, {max_female=}")

difference = max_male - max_female

print(f"{difference=}")

max_male=6300.0, max_female=5200.0
difference=1100.0


### Esercizio 6

3. Dal dataset **dipendenti.csv**, quanti sono i dipendenti che hanno età minore o uguale a 25 anni?

- Qual è il loro salario medio?
- Chi percepisce il salario minore?
- Chi percepisce il salario maggiore?

In [ ]:
path_dip_csv =  "https://github.com/Cinofix/analisi-e-rappresentazione-dati/raw/main/data/dipendenti.csv"

data = pd.read_csv(path_dip_csv)

In [ ]:
data_eta_25 = data[data["Età"] <= 25]
data_eta_25.shape

(193, 6)

In [ ]:
data_eta_25["Salario"].mean()

1985.1657142857143

In [ ]:
index_min = data_eta_25["Salario"].argmin()

data_eta_25.iloc[index_min]

ID            853235
Nome           Delia
Età             20.0
Regione        Lazio
Salario       1336.0
Verificato        SI
Name: 515, dtype: object

In [ ]:
index_max = data_eta_25["Salario"].argmax()

data_eta_25.iloc[index_max]

ID              816931
Nome          Abdellah
Età               22.0
Regione         Veneto
Salario         2695.0
Verificato          NO
Name: 547, dtype: object

### Esercizio 7

4. Utilizzando il dataset **dipendenti.csv**, aggiungere 150€ al salario dei dipedenti che hanno un'età minore o uguale a 25 anni ed un salario minore di 1500€.

In [ ]:
path_dip_csv =  "https://github.com/Cinofix/analisi-e-rappresentazione-dati/raw/main/data/dipendenti.csv"

data = pd.read_csv(path_dip_csv)

mask_query = ((data["Salario"]<1500) & (data["Età"] <=25))

print(data[mask_query]["Salario"].head())

data.loc[mask_query, "Salario"] = data[mask_query]["Salario"]+150

print(data[mask_query]["Salario"].head())

37     1471.0
50     1463.0
95     1360.0
215    1480.0
240    1393.0
Name: Salario, dtype: float64
37     1621.0
50     1613.0
95     1510.0
215    1630.0
240    1543.0
Name: Salario, dtype: float64


---

# Gestire le colonne del dataframe

Un'altra cosa molto utile da sapere quando lavoriamo con i dataframe è come gestire le colonne, in particolare:
- Crearne di nuove
- Rinominarle
- Eliminarle

## Creare una nuova colonna

Creare una nuova colonna è molto semplice, infatti basterà utilizzare la seguente sintassi `data["nome nuova colonna"] = nuovi_dati`.

Ad esempio, se vogliamo creare una nuova colonna con il Salario annuo dei dipendenti lo possiamo fare nel seguente modo:

In [ ]:
import pandas as pd
path_dip_csv = "https://github.com/Cinofix/analisi-e-rappresentazione-dati/raw/main/data/dipendenti.csv"
data = pd.read_csv(path_dip_csv)


mensilità = 13
data["Salario_Annuale"] = data["Salario"] * mensilità

data.head(3)

,ID,Nome,Età,Regione,Salario,Verificato,Salario_Annuale
0,829744,Elona,NaN,Sicilia,1921.0,NO,24973.0
1,800269,Cheikh,53.0,Abruzzo,1563.0,SI,20319.0
2,823929,Girolama,30.0,Emilia Romagna,2144.0,SI,27872.0


Per creare nuove colonne possiamo anche utilizzare delle funzioni personalizzate ed usare il metodo [.apply()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html).

Il metodo `apply()` è un meccanismo versatile per applicare una funzione a dati in un DataFrame o in una Serie. Può essere utilizzato per eseguire trasformazioni complesse sui dati, applicando funzioni personalizzate, funzioni integrate di Python o funzioni lambda.

La funzione `determina_classe` valuta il salario e restituisce la classe corrispondente. Poi, applichiamo questa funzione alla colonna "Salario" utilizzando il metodo apply, creando così la nuov colonna "Classe" nel DataFrame.




In [ ]:
# Definizione della funzione per assegnare la classe
def determina_classe(salario):
    if salario > 2000:
        return 'Alto'
    elif salario >= 1500:
        return 'Medio'
    else:
        return 'Basso'

# Applicazione della funzione alla colonna 'Salario' per creare la nuova colonna 'Classe'
data['Classe'] = data['Salario'].apply(determina_classe)

data.head(3)

,ID,Nome,Età,Regione,Salario,Verificato,Salario_Annuale,Classe
0,829744,Elona,NaN,Sicilia,1921.0,NO,24973.0,Medio
1,800269,Cheikh,53.0,Abruzzo,1563.0,SI,20319.0,Medio
2,823929,Girolama,30.0,Emilia Romagna,2144.0,SI,27872.0,Alto


## Rinominare una colonna

Nel caso in cui dobbiamo rinominare una o più colonne possiamo utilizzare il metodo [.rename()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rename.html) specificando nel parametro `columns=` un dizionario che mappa il vecchio nome con il nuovo.

Ad esempio, vogliamo rinominare la nuova colonna `Salario_Annuale` in `Salario_Lordo_Annuo` lo possiamo fare nel seguente modo:



In [ ]:
rename_columns = {"Salario_Annuale" : "Salario_Lordo_Annuo"}

data.rename(columns = rename_columns, inplace = True)

data.head(3)

,ID,Nome,Età,Regione,Salario,Verificato,Salario_Lordo_Annuo,Classe
0,829744,Elona,NaN,Sicilia,1921.0,NO,24973.0,Medio
1,800269,Cheikh,53.0,Abruzzo,1563.0,SI,20319.0,Medio
2,823929,Girolama,30.0,Emilia Romagna,2144.0,SI,27872.0,Alto


**TIPS AND TRICKS**: il metodo `.rename()`, come tanti altri metodi, contiene il parametro `inplace=` che, se impostato a `True`, modifica il daframe stesso che stiamo utilizzando.

## Eliminare una colonna

L'ultima operazione utile è poter eliminare una o più colonne.

Eseguiamo questa operazione utilizzando il metodo [.drop()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html) specificando il nome della colonna o una *lista* di colonne che vogliamo eliminare.


Ad esempio eliminiamo da `data` la colonna `Salario_Lordo_Annuo` che abbiamo precedentemente craeto.

In [ ]:
data_drop_column = data.drop("Salario_Lordo_Annuo", axis = 1)

data_drop_column.head(3)

,ID,Nome,Età,Regione,Salario,Verificato,Classe
0,829744,Elona,NaN,Sicilia,1921.0,NO,Medio
1,800269,Cheikh,53.0,Abruzzo,1563.0,SI,Medio
2,823929,Girolama,30.0,Emilia Romagna,2144.0,SI,Alto


Anche qui possiamo utilizzare il parametro `inplace=True`.

In [ ]:
data.drop("Salario_Lordo_Annuo", axis = 1, inplace = True)

data.head()

,ID,Nome,Età,Regione,Salario,Verificato,Classe
0,829744,Elona,NaN,Sicilia,1921.0,NO,Medio
1,800269,Cheikh,53.0,Abruzzo,1563.0,SI,Medio
2,823929,Girolama,30.0,Emilia Romagna,2144.0,SI,Alto
3,842108,Marwa,60.0,Umbria,1798.0,NO,Medio
4,861546,NaN,26.0,Toscana,2192.0,SI,Alto


## Esercizi


In [ ]:
import pandas as pd
path_dip_csv = "https://github.com/Cinofix/analisi-e-rappresentazione-dati/raw/main/data/dipendenti.csv"
data = pd.read_csv(path_dip_csv)

#### Esercizio 8
Aggiungi una nuova colonna chiamata "Stipendio_Orario" al DataFrame che rappresenta il salario orario di ciascun dipendente (considerando 40 ore di lavoro settimanale).

#### Esercizio 9
Aggiungi una nuova colonna chiamata "Classe_Etaria" che categorizzi gli individui in base all'età: "Giovane" per età inferiore a 30, "Adulto" per età compresa tra 30 e 50, e "Anziano" per età superiore a 50.

#### Esercizio 10
Prendi i dipendenti con un reddito inferiore a 1300€ e aggiungi loro 150€.

#### Esercizio 11
Rinomina la colonna "Regione" in "Provenienza" e la colonna "ID" in "Matricola" nel DataFrame. Utilizzare solo una volta la funzione rename.

#### Esercizio 12
Calcola l'imposta sul reddito per ogni dipendente secondo le seguenti regole:

- Nessuna imposta per redditi fino a 1500€
- 10% di imposta per redditi compresi tra 1500€ e 2500€
- 20% di imposta per redditi superiori a 2500€

#### Esercizio 13
Elimina le colonne "Età", "Salario" e "Stipendio_Orario", utilizza solo una volta la funzione drop.


---


# Gestire i dati che mancano
Non sempre i file csv che troviamo online hanno tutti i valori inseriti.

Ad esempio il file "dipendenti.csv" ha dei valori nulli, questo lo possiamo osservare dalla colonna **Non-Null Count** quando utilizziamo il metodo `.info()` o contiamo quanti valori sono **NaN** utilizzando insieme i metodi [.isnull()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isnull.html) e `.sum()`.

In [ ]:
import pandas as pd
path_dip_csv = "https://github.com/Cinofix/analisi-e-rappresentazione-dati/raw/main/data/dipendenti.csv"
data = pd.read_csv(path_dip_csv)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ID          1000 non-null   int64  
 1   Nome        977 non-null    object 
 2   Età         928 non-null    float64
 3   Regione     854 non-null    object 
 4   Salario     905 non-null    float64
 5   Verificato  949 non-null    object 
dtypes: float64(2), int64(1), object(3)
memory usage: 47.0+ KB


In [ ]:
data.isnull().sum()

ID              0
Nome           23
Età            72
Regione       146
Salario        95
Verificato     51
dtype: int64

Inoltre, se vogliamo visualizzare soltanto le righe dove manca l'informazione relativa al **Salario** dei dipendenti possiamo creare una *maschera* utilizzare il metodo [.isna()](https://pandas.pydata.org/docs/reference/api/pandas.Series.isna.html).

In [ ]:
data[data["Salario"].isna()]

,ID,Nome,Età,Regione,Salario,Verificato
19,897435,Rita rosa,37.0,Lombardia,NaN,NO
25,894611,Franca angela,31.0,Sicilia,NaN,NO
32,872407,Morgana,33.0,Abruzzo,NaN,SI
43,826433,Barbara,61.0,Liguria,NaN,NO
53,896046,Ottavio,23.0,Valle d'Aosta,NaN,NO
...,...,...,...,...,...,...
891,813696,Mehdi,29.0,Toscana,NaN,NO
918,807356,John,22.0,Piemonte,NaN,NO
935,862770,Anna,55.0,Abruzzo,NaN,NO
949,827424,Marioara,39.0,NaN,NaN,SI


## Come risolvere il problema ?

Abbiamo osservato che esistono dei dati mancanti, come si potrebbe agire per risolvere il problema?

### Soluzione 1: rimuovere i dati con **.dropna()**

Questa prima soluzione prevede di eliminare completamente le righe dove i dati mancano tramite l'utilizzo del metodo [.dropna()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html).

In [ ]:
df_drop_na = data.dropna()

df_drop_na.info()

<class 'pandas.core.frame.DataFrame'>
Index: 668 entries, 1 to 997
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ID          668 non-null    int64  
 1   Nome        668 non-null    object 
 2   Età         668 non-null    float64
 3   Regione     668 non-null    object 
 4   Salario     668 non-null    float64
 5   Verificato  668 non-null    object 
dtypes: float64(2), int64(1), object(3)
memory usage: 36.5+ KB


Di base, il metodo `.dropna()` elimina tutte le righe dove è presente **almeno un** valore nullo.

Può capitare di essere interessati ad eliminare le righe dove i valori nulli sono presenti in un `subset` di una o più colonne, facendo attenzione che rimarrebbero comunque eventuali valori nulli nelle altre colonne.

In [ ]:
data.dropna(subset=["Salario"]).info()

<class 'pandas.core.frame.DataFrame'>
Index: 905 entries, 0 to 999
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ID          905 non-null    int64  
 1   Nome        883 non-null    object 
 2   Età         837 non-null    float64
 3   Regione     772 non-null    object 
 4   Salario     905 non-null    float64
 5   Verificato  859 non-null    object 
dtypes: float64(2), int64(1), object(3)
memory usage: 49.5+ KB


### Soluzione 2: sostituire i valori mancanti con **.fillna()**

Nel caso eliminare le righe con valori nulli riducesse di molto la dimensione del nostro dataset, potrebbe risulare utile rimpiazzare tali valori con qualcos'altro.

In particolare, il metodo [.fillna(`value`)](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html#pandas.DataFrame.fillna) ci permette di fare esattamente questo.

In [ ]:
data_fill_na = data.fillna(0)
print(data_fill_na.head(3))
print("-"*70)
print(data_fill_na.tail(3))

       ID      Nome   Età         Regione  Salario Verificato
0  829744     Elona   0.0         Sicilia   1921.0         NO
1  800269    Cheikh  53.0         Abruzzo   1563.0         SI
2  823929  Girolama  30.0  Emilia Romagna   2144.0         SI
----------------------------------------------------------------------
         ID         Nome   Età   Regione  Salario Verificato
997  886127  Carla maria  23.0  Campania   2149.0         NO
998  847566     El mehdi  32.0         0   1817.0         NO
999  891216        Suada  23.0         0   2623.0         NO


In [ ]:
print(data_fill_na.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ID          1000 non-null   int64  
 1   Nome        1000 non-null   object 
 2   Età         1000 non-null   float64
 3   Regione     1000 non-null   object 
 4   Salario     1000 non-null   float64
 5   Verificato  1000 non-null   object 
dtypes: float64(2), int64(1), object(3)
memory usage: 47.0+ KB
None


Però non sempre sostituire tutti NaN con lo stesso valore va bene. Ad esempio, impostare i valori di "Età" o "Salario" a 0 potrebbe avere un senso, invece "Regione" ne ha già meno.

Per risolvere questo problema possiamo passare al parametro `value` di `.fillna()` un dizionario per specificare che valore vogliamo sosituire per ogni colonna.

In [ ]:
fill_values = {"Nome":"Sconosciuto", "Età": 33,
               "Regione": "Non registrata", "Salario": 1450, "Verificato":"NO"}

maschera_nome_nullo = data["Nome"].isna()

data_fill_na = data.fillna(value=fill_values)

print(data[maschera_nome_nullo].head(3))
print("-"*70)
print(data_fill_na[maschera_nome_nullo].head(3))

         ID Nome   Età         Regione  Salario Verificato
4    861546  NaN  26.0         Toscana   2192.0         SI
72   878054  NaN  48.0  Emilia Romagna   1642.0         SI
114  867706  NaN  34.0       Lombardia      NaN         SI
----------------------------------------------------------------------
         ID         Nome   Età         Regione  Salario Verificato
4    861546  Sconosciuto  26.0         Toscana   2192.0         SI
72   878054  Sconosciuto  48.0  Emilia Romagna   1642.0         SI
114  867706  Sconosciuto  34.0       Lombardia   1450.0         SI


## Esercizi:

### Esercizio 14
Sostituire i valori numerici nulli di "Salario" ed "Età" con il relativo valore medio della colonna utilizzando fillna. Salvare il risultato in una variabile chiamata "data_sal_eta_ok".

### Esercizio 15

Caricare il file csv "dipendenti2.csv" e rimuovere tutti i dipendenti senza nome, dopodichè creare una nuova colonna "Buono_pasto" con costo pari ad 1 ora lavorativa.

Dopodichè controllare se la colonna "Buono_pasto" ha dei valori nulli, nel caso sostiturili con il valore del buono pasto meno costoso.